In [1]:
import pyodbc
import sys
import datetime

from Activo import *
from Accion import *
from Bono import *
from Derivado import *
from DerivadosTipos.DerivadosSCC import *
from extraer_accion import *

from Cartera import *

In [2]:
server = '172.16.1.38'
username = 'sa'
password = 'qwerty123'
driver = '{ODBC Driver 17 for SQL Server}'
cn = pyodbc.connect('DRIVER=' + driver + ';SERVER=' + server + ';UID=' + username + ';PWD=' + password)

In [3]:
def cambiar_fecha(fecha):

    fechas = fecha.split("-")
    fechas = fechas[2].split(" ")[0] + "/" + fechas[1] + "/" + fechas[0]
    return fechas

In [4]:
def bonos_dbAlgebra(n,cn):
    bonos = 'SELECT TOP(' + str(n) + ") bono.Moneda, bono.TablaDesarrollo, bono.Base1 , bono.Base2, bono.FechaEmision, riesgo.Riesgo AS RiesgoInt FROM [dbAlgebra].[dbo].[TdNemoRF] AS bono , [dbAlgebra].[dbo].[VwRiesgoRF] AS riesgo WHERE bono.Nemotecnico = riesgo.Nemotecnico"
    bonos = pd.read_sql(bonos, cn)
    return pd.DataFrame(bonos)

def derivados_dBDerivados(cantidad, cn):

    derivados = "SELECT TOP ("+ str(cantidad) +") * FROM [dbDerivados].[dbo].[TdCarteraDerivados_V2]"
    derivados = pd.read_sql(derivados, cn)

    derivado = pd.DataFrame()
    dev = []

    for i in range(cantidad):

        info_derivado = dict()
        info_derivado["Tipo"] = derivados["Tipo"][i]
        info_derivado["ID_Key"] = derivados["ID_Key"][i]
        info_derivado["Administradora"] = derivados["Administradora"][i]
        info_derivado["Fondo"] = derivados["Fondo"][i]
        info_derivado["Contraparte"] = derivados["Contraparte"][i]
        info_derivado["ID"] = int(derivados["ID"][i])
        info_derivado["Nemotecnico"] = derivados["Nemotecnico"][i]
        info_derivado["Mercado"] = derivados["Mercado"][i]     
        fecha = datetime.date(2017, 12, 20)
        hora = '1700'
        info_derivado["FechaEfectiva"] = cambiar_fecha(str(derivados["FechaEfectiva"][i]))
        info_derivado["FechaVenc"] = cambiar_fecha(str(derivados["FechaVenc"][i]))
        info_derivado["AjusteFeriados"] = derivados["AjusteFeriados"][i]
        info_derivado["NocionalActivo"] = derivados["NocionalActivo"][i]
        info_derivado["MonedaActivo"] = derivados["MonedaActivo"][i]
        info_derivado["MonedaBase"] = derivados["MonedaBase"][i]
        info_derivado["TipoTasaActivo"] = derivados["TipoTasaActivo"][i]
        info_derivado["TipoTasaPasivo"] = derivados["TipoTasaPasivo"][i]
        info_derivado["TasaActivo"] = derivados["TasaActivo"][i]
        info_derivado["TasaPasivo"] = derivados["TasaPasivo"][i]
        info_derivado["FrecuenciaActivo"] = derivados["FrecuenciaActivo"][i]
        info_derivado["FrecuenciaPasivo"] = info_derivado["FrecuenciaActivo"]

        info1 = pd.DataFrame([info_derivado])
        derivado_info = DerivadosSCC(fecha, hora, info1, cn)

        dev.append(derivado_info)

    derivado["Derivado"] = dev
    return derivado

In [5]:
def conversionSYP(riesgo):
    return {1: 'AAA',2: 'AA',3: 'AA',4: 'AA',5: 'A',6: 'A',7: 'A',8: 'BBB',9: 'BBB',10: 'BBB',\
            11: 'BB',12: 'BB',13: 'BB',14: 'B',15: 'B',16: 'B',17: 'CCC',18: 'CC',19: 'CC',20: 'C',\
            21: 'C',22: 'C',23: 'D',24: 'E'}.get(riesgo)

def ajustes(bonos):
    bonos['FechaEmision'] = bonos['FechaEmision'].apply(lambda x: x.strftime('%Y-%m-%d'))
    bonos['Base1'] = bonos['Base1'].apply(lambda x: x if x != -1 else 'ACT')
    bonos['Base2'] = bonos['Base2'].apply(lambda x: x if x != -1 else 'ACT')
    bonos['Convencion'] = bonos['Base1'].apply(lambda x: str(x)) + '/' + bonos['Base2'].apply(lambda x: str(x))
    bonos['Riesgo'] = bonos['RiesgoInt'].apply(lambda x: conversionSYP(x))
    return bonos

In [6]:
#bonos = ajustes(bonos_dbAlgebra(1,cn))
#bonos

In [7]:
derivados = derivados_dBDerivados(5, cn)
derivados

C:\Users\groso\Desktop\PracticaV2\Practica\Intento\Cartera\DerivadosTipos\DerivadosSCC.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  info_cartera.MonedaActivo[0] = "CLP"


,Derivado
0,<DerivadosTipos.DerivadosSCC.DerivadosSCC obje...
1,<DerivadosTipos.DerivadosSCC.DerivadosSCC obje...
2,<DerivadosTipos.DerivadosSCC.DerivadosSCC obje...
3,<DerivadosTipos.DerivadosSCC.DerivadosSCC obje...
4,<DerivadosTipos.DerivadosSCC.DerivadosSCC obje...


In [8]:
acciones = historico("DE000DWS2D90", "A", 60)


         Fecha Fondo   Nemotecnico   Nominales  ValorizacionCLP          Tasa  \
0   2017-08-28     A  DE000DWS2D90  15139.1281     4.597026e+09  303652.00000   
1   2017-08-29     A  DE000DWS2D90  15139.1281     4.542262e+09  300034.62500   
2   2017-08-30     A  DE000DWS2D90  15139.1281     4.428379e+09  292512.15625   
3   2017-08-31     A  DE000DWS2D90  15139.1281     4.446764e+09  293726.59375   
4   2017-09-01     A  DE000DWS2D90  15139.1281     4.497070e+09  297049.50000   
..         ...   ...           ...         ...              ...           ...   
603 2020-02-06     A  DE000DWS2D90  29580.2782     1.195760e+10  404242.18750   
604 2020-02-07     A  DE000DWS2D90  29580.2782     1.205133e+10  407410.87500   
605 2020-02-10     A  DE000DWS2D90  29580.2782     1.207114e+10  408080.81250   
606 2020-02-11     A  DE000DWS2D90  29580.2782     1.212237e+10  409812.43750   
607 2020-02-12     A  DE000DWS2D90  29580.2782     1.224062e+10  413810.12500   

     Duracion  Compras  Ven

In [9]:
cartera = Cartera(acciones, pd.DataFrame(), pd.DataFrame(),'CLP', datetime.date(2019,2,1), cn, 60)

In [10]:
print(cartera.get_historicos_totales())

     DE000DWS2D90
0        0.000000
1       -0.011984
2       -0.025392
3        0.004143
4        0.011249
..            ...
603      0.012504
604      0.007808
605      0.001643
606      0.004234
607      0.009708

[608 rows x 1 columns]


In [11]:
print(cartera.get_retornos_totales())

     DE000DWS2D90
0        0.000000
1       -0.011984
2       -0.025392
3        0.004143
4        0.011249
..            ...
603      0.012504
604      0.007808
605      0.001643
606      0.004234
607      0.009708

[608 rows x 1 columns]


In [12]:
cartera.set_volatilidad_cartera()
print(cartera.get_volatilidad_cartera())

0.010031064672316407
